# Amplitude Amplification Operators

Present notebook reviews the *Amplitude Amplification* (**AA/**) module, from present library, where all the mandatory functions for creating a Grover-like operator ($Q$) are developed.

Present notebook and module are based on the following references:

* *Brassard, G., Hoyer, P., Mosca, M., & Tapp, A. (2000). Quantum amplitude amplification and estimation.AMS Contemporary Mathematics Series, 305. https://arxiv.org/abs/quant-ph/0005055v1*
* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
import sys
sys.path.append("../")

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from libraries.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from libraries.utils.data_extracting import get_results

## 1. Data loading

Before doing any amplification we want to load some data into the quantum circuit, as this step is only auxiliary to see the efect of an amplification, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$
Note that this probability distribution is properly normalised. For that purpose we will use the function *load_probability*. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from libraries.DL.data_loading import load_probability, load_array

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
p = x/np.sum(x)

In [ ]:
probability_routine = qlm.QRoutine()
register = probability_routine.new_wires(n)
probability_routine.apply(load_probability(p),register)
%qatdisplay probability_routine --svg

In [ ]:
results_loading, _, _, _ = get_results(probability_routine, linalg_qpu=linalg_qpu, shots=0)
amplitudes_loading = results_loading["Amplitude"].values
results_loading

In [ ]:
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_loading)
print('Test OK?: ',np.isclose(np.sqrt(p), amplitudes_loading).all())

See that the information stored in the quantum state is the same as the one stored in array $p$. For more information about loading data into the quantum circuit see the notebook *data_loading_use*.

## 2. Reflections

The next mandatory ingredient for amplification are **reflections**. A reflection rotates the phase of a state $\dfrac{\pi}{2}$. In other words, for real numbers **it changes the sign of the state**. 

The function *reflection* from **AA/amplitude_amplification** module is used for creating **reflections**.Let's do some examples.

In [ ]:
from libraries.AA.amplitude_amplification import reflection

### 2.1 First example

We will start with a very simple example, in this case we want to flip sign of state $|7\rangle$. For that we will use the function *reflection*. This function takes a list as a parameter, the list specifies which combination of qubits is going to be reflected. In this case, the binary representation of the state $|7>$ is $111$, so the argument that we have to pass to the function is $[1,1,1]$.

In [ ]:
reflection1 = qlm.QRoutine()
reflection1_register = reflection1.new_wires(n)
reflection1.apply(probability_routine,reflection1_register)
reflection1.apply(reflection([1,1,1]),reflection1_register)
%qatdisplay reflection1 --depth 0 --svg

Now we print the amplitudes stored in the quantum circuit against the ones stored in the array $p$ to see the differences in sign.

In [ ]:
results_reflection1, _, _, _ = get_results(reflection1, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection1 = results_reflection1["Amplitude"].values
results_reflection1

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection1)

In [ ]:
print('Test OK: ',
    np.isclose(np.sqrt(p)[:-1], amplitudes_reflection1[:-1]).all() and
    np.isclose(np.sqrt(p)[-1], -amplitudes_reflection1[-1]).all()
     )

As we see the last state has changed from $0.5$ to $-0.5$.

## 2.2 Second example

In this example we want to do something more difficult. We want flip the sign of all states that start with one. Those states are $00\mathbf{1}\longrightarrow |1\rangle$, $01\mathbf{1}\longrightarrow |3\rangle$, $10\mathbf{1}\longrightarrow |5\rangle$ and $11\mathbf{1}\longrightarrow |7\rangle$. Again, we have to use the funciton *reflection*. This time we have to apply the reflection to the first qubit, so we just apply to the first register.

In [ ]:
reflection2 = qlm.QRoutine()
reflection2_register = reflection2.new_wires(n)
reflection2.apply(probability_routine,reflection2_register)
reflection2.apply(reflection([1]),reflection2_register[0])
%qatdisplay reflection2 --svg

Now we will see that the positions $1,3,5$ and $7$ have a different sign from the ones in array $p_d$.

In [ ]:
results_reflection2, _, _, _ = get_results(reflection2, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection2 = results_reflection2["Amplitude"].values
results_reflection2

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection2)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[[0, 2, 4, 6]],
        amplitudes_reflection2[[0, 2, 4, 6]]
    ).all() and
    np.isclose(
        np.sqrt(p)[[1, 3, 5, 7]],
        -amplitudes_reflection2[[1, 3, 5, 7]]
    ).all()
)

### 2.3 Third example

In this example we will do a reflection in a more specific state. In this case, we want to do the reflection of the states $|1\rangle,|3\rangle,|5\rangle,|6\rangle,|7\rangle$. We can divide the process of the reflection in two sub reflections.
- First, note that the reflection over states $|1\rangle,|3\rangle,|5\rangle,|7\rangle$ is the one implemented in Section 2.2.
- Second, the remaining state $|6\rangle$ can be reflected using the same strategy as in Section 2.1.

In [ ]:
reflection3 = qlm.QRoutine()
reflection3_register = reflection3.new_wires(n)
reflection3.apply(probability_routine,reflection3_register)
reflection3.apply(reflection([1]),reflection3_register[0])
reflection3.apply(reflection([1,1,0]),reflection3_register)
%qatdisplay reflection3 --depth 0 --svg

Now we will see that the positions $1,3,5,6$ and $7$ have a different sign from the ones in array $p_d$.

In [ ]:
results_reflection3, _, _, _ = get_results(reflection3, linalg_qpu=linalg_qpu, shots=0)
amplitudes_reflection3 = results_reflection3["Amplitude"].values
results_reflection3

In [ ]:
print("Classical amplitudes: ", np.sqrt(p))
print("Quantum amplitudes: ",amplitudes_reflection3)

In [ ]:
print('Test OK: ',
    np.isclose(
        amplitudes_reflection3[[1, 3, 5, 6, 7]],
        -np.sqrt(p)[[1, 3, 5, 6, 7]]
    ).all() and
    np.isclose(
        amplitudes_reflection3[[0, 2, 4]],
        np.sqrt(p)[[0, 2, 4]]
    ).all()
)

Note that doing consecutive reflections over sets is a conmutative operation **when the sets are disjoint**.

## 3. Grover operator

In general, the Grover operator works in the following way. Let's say that we have an operator (a routine) that performs the following operation:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$
where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states. Now, the Grover operator $\mathcal{G}$ does the following transformation:
$$ |\Psi \rangle \longrightarrow \mathcal{G}^k|\Psi\rangle= \sin\left((2k+1)\theta\right)|\Psi_0\rangle +\cos\left((2k+1)\theta\right)|\Psi_1\rangle.$$ 
This operator is known in the literature as **amplitude amplification** as, in principle, it *increases* the probability of obtaining the state $|\phi\rangle$. Note that, when the angle $(2k+1)\theta$ goes over $\dfrac{\pi}{2}$, the probability instead of increase starts decreasing. Whenever we use the term amplification keep in mind that it can have this effect.

The Grover operator $\mathcal{G}$ can be decomposed in 2 different operators:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Where $\hat{U}_{|\Psi\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ are:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$

In this section we are going to review all the operators programed for creating the Grover operator.



### 3.2 Operator $\hat{U}_{|\Psi_{0}\rangle}$

First mandatory operator is:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

When we apply this operator on state $|\Psi\rangle$:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle = -\sin(\theta)|\Psi_{0}\rangle+\cos(\theta)|\Psi_{1}\rangle$$

So operator $\hat{U}_{|\Psi_{0}\rangle }$ is a reflection of the state $|\Psi_{0}\rangle$ around state $|\Psi_{1}\rangle$. Or in a graphic view:

![title](images/OraculeReflection.png)


Abstract Gate *U0* from **AA/amplitude_amplification** module allow us do the operation. We need to provide as input which is the state that we want to flip the sign. This is done with three arguments. The first one is always the oracle. The second one is a list containing the binary representation of the state that we want to mark. The third one is a list of the registers over which we want to act. In this sense, the syntax is very similar to that of the reflections. Indeed, it is implemented using the function *reflection*. Next we give two examples.

In [ ]:
from libraries.AA.amplitude_amplification import U0

#### 3.2.1 First example

In this first example we want to mark the state $|7\rangle$. It's binary representation is $111$, and we want to act on the register [0,1,2]. The second input for the gate *U0* is [1,1,1] and the third input is [0,1,2].

In [ ]:
routine_U0_1 = qlm.QRoutine()
register_U0_1 = routine_U0_1.new_wires(probability_routine.arity)
routine_U0_1.apply(probability_routine,register_U0_1)
routine_U0_1.apply(U0(probability_routine,[1,1,1],[0,1,2]),register_U0_1)
%qatdisplay routine_U0_1 --svg

If we display the circuit with one more layer of depth, we will see that it is implemented as a reflection([1,1,1]) acting on the register [0,1,2].

Last, we will show the amplitudes stored in the quantum circuit and we will see that only the last element is affected

In [ ]:
results_U0_1,_ ,_ ,_  = get_results(routine_U0_1,linalg_qpu)

In [ ]:
amplitudes_U0_1 = results_U0_1["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U0_1)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[:-1],
        amplitudes_U0_1[:-1]
    ).all() and
    np.isclose(
        np.sqrt(p)[-1],
        -amplitudes_U0_1[-1]
    ).all()
)

#### 3.2.1 Second example

For the second example we want flip the sign/mark all states that start with one. Those states are $00\mathbf{1}\longrightarrow |1\rangle$, $01\mathbf{1}\longrightarrow |3\rangle$, $10\mathbf{1}\longrightarrow |5\rangle$ and $11\mathbf{1}\longrightarrow |7\rangle$. Again, we have to use the funciton *U0*, but this time we indicate that we are only acting upon the first register.

In [ ]:
routine_U0_2 = qlm.QRoutine()
register_U0_2 = routine_U0_2.new_wires(probability_routine.arity)
routine_U0_2.apply(probability_routine,register_U0_1)
routine_U0_2.apply(U0(probability_routine,[1],[0]),register_U0_2)
%qatdisplay routine_U0_2 --svg

If we display the circuit with one more layer of depth, we will see that it is implemented as a *reflection([1])* acting on the register *[0]*.

Last, we will show the amplitudes stored in the quantum circuit and we will see that only states $|1\rangle$, $|3\rangle$, $|5\rangle$ and $|7\rangle$ are affected.

In [ ]:
results_U0_2,_ ,_ ,_  = get_results(routine_U0_2,linalg_qpu)

In [ ]:
amplitudes_U0_2= results_U0_2["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U0_2)

In [ ]:
print('Test OK: ',
    np.isclose(
        np.sqrt(p)[[0, 2, 4, 6]],
        amplitudes_U0_2[[0, 2, 4, 6]]
    ).all() and
    np.isclose(
        np.sqrt(p)[[1, 3, 5, 7]],
        -amplitudes_U0_2[[1, 3, 5, 7]]
    ).all()
)

**Note that we cannot use operator *U0* to mark states that require more than one reflection. This is the case for example of the set of states $|1\rangle,|3\rangle,|5\rangle,|6\rangle,|7\rangle$.**

 ### 3.2 Operator $\hat{U}_{|\Psi}\rangle$

The $\hat{U}_{|\Psi\rangle}$ (**diffusor**):
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|,$$
is a reflection of state $|\Psi\rangle$ around the state $|\Psi\rangle^{\perp}$ (where $|\Psi\rangle^{\perp} \perp |\Psi\rangle$).

Using the graphic representation:

![title](images/StateReflection.png)

So:

$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = \hat{I}|\Psi\rangle - 2|\Psi\rangle \langle \Psi|\Psi\rangle = |\Psi\rangle - 2|\Psi\rangle = -|\Psi\rangle$$


For implementing this operator function *U* from **AA/amplitude_amplification** module will be used. It only needs as input the oracle


In [ ]:
from libraries.AA.amplitude_amplification import U

In [ ]:
routine_U = qlm.QRoutine()
register_U = routine_U.new_wires(probability_routine.arity)
routine_U.apply(probability_routine,register_U)
routine_U.apply(U(probability_routine),register_U)
%qatdisplay routine_U --svg

When we show the results we will see that all the amplitudes have changed their sign.

In [ ]:
results_U,_ ,_ ,_  = get_results(routine_U,linalg_qpu)

In [ ]:
amplitudes_U = results_U["Amplitude"].values
print("Classical probabilities: ", np.sqrt(p))
print("Quantum probabilities: ",amplitudes_U)

In [ ]:
print('Test OK: ', np.isclose(np.sqrt(p), -amplitudes_U).all())

### 3.3 Grover

Now we have all ingredients to create a grover operator:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

The grover operator can be implemented by function *grover* from **AA/amplitude_amplification**. Let's see som examples

In [ ]:
from libraries.AA.amplitude_amplification import grover

### 3.3.1 First example

In our first example we are going to amplify the probability of obtaining the state $|1\rangle$.

In [ ]:
grover1 = qlm.QRoutine()
grover1_register = grover1.new_wires(n)
grover1.apply(probability_routine,grover1_register)

grover1.apply(grover(probability_routine,[0,0,1],[0,1,2]),grover1_register)
%qatdisplay grover1 --depth 0 --svg

The critial part is *grover(probability_routine,[0,0,1],[0,1,2])*.
The first argument of the function is always the routine that we are going to amplify. 
In the second argument we have to specify our target state for amplification. In this case, as we want to amplify the state $|1\rangle$ we use its binary representation $001$.
Last, the third input is an index whcih specifies over which part of the quantum register we want to act. In this case, as the three quantum registers are involved, we put the three of them $[0,1,2]$. In the next examples it will become clearer how it works.

Now we measure the amplified probabilities

In [ ]:
results_grover1,_ ,_ ,_  = get_results(grover1,linalg_qpu)
probabilities_grover1 = results_grover1["Probability"].values
probabilities_grover1

Let's explain what we have done so far. In our example, we do the following identifications:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \cos(\theta)|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
When we do the amplification and measure the probabilities we will obtain in position $1$ the new probability $p_{a} = \sin^2(3\theta)$. The original probability was $p_o = \sin^2(\theta)$. To check if the amplified probability and the original probability are correctly computed, we simply use the fact that, we can recover $\theta$ from the amplified probability doing the operation:
$$ \theta = \dfrac{\arcsin\left(\sqrt{p_a}\right)}{3}.$$
The we relate it with the original one substituting the angle, hence:
$$
p_o = \sin ^2\left(\dfrac{\arcsin\left(\sqrt{p_a}\right)}{3}\right)
$$
In the next cell we check that the amplification is done correctly

In [ ]:
unamplified_grover1 = np.sin(np.arcsin(np.sqrt(probabilities_grover1[1]))/3)**2
print("Original probabilities: ", p[1])
print("Quantum probabilities: ",unamplified_grover1)

### 3.2 Second example

In this second example we are going to show how to amplify a more complex state. Say that now we want to amplify the state composed by $|0\rangle,|1\rangle,|2\rangle,|3\rangle$. Now we will see the correspondence of this new setup with the Grover setup:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\phi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle\right].\\
    & \cos(\theta)|\phi^\dagger\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$
In binary representation, the states $|0\rangle,|2\rangle,|4\rangle,|6\rangle$ are $|\mathbf{0}00\rangle,|\mathbf{0}01\rangle,|\mathbf{0}10\rangle,|\mathbf{0}11\rangle$ respectively. Here we are interested in the join probability of getting this states, this is the same as asking for the probabiliy of the leftmost qubit being zero.
Before doing an amplification we are going to do the unamplified version.

In [ ]:
grover2 = qlm.QRoutine()
grover2_register = grover2.new_wires(n)
grover2.apply(probability_routine,grover2_register)
%qatdisplay grover2 --depth 0 --svg

In [ ]:
results_grover2,_,_,_ = get_results(grover2,linalg_qpu = linalg_qpu,qubits = [n-1])
results_grover2

When we only measure the leftmost qubit ($q_2$) we are effectively performing the following operation:
$$ 
    \begin{array}{l}
p_{|0\rangle} = \sin^2\left(\theta\right) = \dfrac{1}{0+1+2+3+4+5+6+7+8} \left[|\sqrt{0}|^2+|\sqrt{1}|^2+|\sqrt{2}|^2+|\sqrt{3}|^2\right].\\
p_{|1\rangle} = \cos^2\left(\theta\right) = \dfrac{1}{0+1+2+3+4+5+6+7+8} \left[|\sqrt{4}|^2+|\sqrt{5}|^2+|\sqrt{6}|^2+|\sqrt{7}|^2\right].\\
\end{array}
$$

In the next cell we check it with the classical probability

In [ ]:
print(" Probability of state |0>: ", p[0]+p[1]+p[2]+p[3])
print(" Probability of state |1>: ", p[4]+p[5]+p[6]+p[7])

Next we will amplify the probability of the state marked with the rightmost qubit being $0$. For that we again use the *grover* function. Apart from the oracle we need to indicate the state that we want to amplify.

In [ ]:
grover2 = qlm.QRoutine()
grover2_register = grover2.new_wires(n)
grover2.apply(probability_routine,grover2_register)
grover2.apply(grover(probability_routine,[0],[n-1]),grover2_register)
%qatdisplay grover2 --svg

Now we will measure again the probabilities of the leftmost qubit ($q_2$) being zero.

In [ ]:
results_grover2,_,_,_ = get_results(grover2,linalg_qpu = linalg_qpu,qubits = [n-1])
results_grover2

We cleary see that we have increased the probability of obtaining $|0\rangle$ at the cost of decreasing the probability of obtaining $|1\rangle$. Again we can see the correspondence of this amplified probability with the unamplified one.

In [ ]:
unamplified_grover2 = np.sin(np.arcsin(\
                    np.sqrt(results_grover2["Probability"].iloc[0]))/3)**2
print("Unamplified probability of state |0>: ", unamplified_grover2)
print("Classical probability of state |0>: ", p[0]+p[1]+p[2]+p[3])
